In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from fuzzywuzzy import fuzz
from sklearn.cluster import AffinityPropagation
import os
import groupby_toolz
import spacy #https://spacy.io/usage/linguistic-features
# install spacy here https://spacy.io/usage
# import en_core_web_sm
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from IPython.display import display
pd.options.display.max_columns = None
from decimal import *
TWOPLACES = Decimal(10) ** -2
# https://stackoverflow.com/questions/21903842/how-do-i-compare-two-strings-in-python

#Colors
import webcolors
import math
from math import sqrt
from collections import Counter
from fractions import Fraction


In [3]:
#######################################################################  Categorizing Syndication Data  ##########################################################################
def col_group(df, fuzzy = 'partial',return_dict = False):
    '''
    Affinity Clustering to sort column names into clusters based on various fuzzy match
    param:
    =======
    df: Pandas DataFrame, syndicated DataFrame
    fuzzy: str, default value is partial, to select fuzzy ratio type: ['ratio','partial','token_sort','token_set','partial_sort','partial_set'] 
    return_dict: bool, if False, prints output. If True, returns a dict with each cluster. Default value False 
    '''
    # Modified from: https://bit.ly/3vRuE1y
    assert isinstance(return_dict, bool), "return_dict argument must be boolean"
    assert fuzzy in ['ratio','partial','token_sort','token_set','partial_sort','partial_set'],\
    "fuzzy must be in 'ratio','partial','token_sort','token_set','partial_sort', or 'partial_set'"
    words = np.asarray(df.columns)
    # Pick a fuzzy ratio
    if fuzzy == 'ratio':
        lev_similarity = np.array([[fuzz.ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'partial':
        lev_similarity = np.array([[fuzz.partial_ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'token_sort':
        lev_similarity = np.array([[fuzz.token_sort_ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'token_set':
        lev_similarity = np.array([[fuzz.token_set_ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'partial_sort':
        lev_similarity = np.array([[fuzz.partial_token_sort_ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'partial_set':
        lev_similarity = np.array([[fuzz.partial_token_set_ratio(w1,w2) for w1 in words] for w2 in words])
    # Affinity clustering
    affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
    affprop.fit(lev_similarity)
    # Simply print output
    if return_dict == False:
        for cluster_id in np.unique(affprop.labels_):
            exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
            cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
            cluster_str = ", ".join(cluster)
            print(" - *%s:* %s" % (exemplar, cluster_str))
            print('\b')
    # Returns a dictionary instead
    elif return_dict == True:
        dict_ = defaultdict()
        for cluster_id in np.unique(affprop.labels_):
            exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
            cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
            cluster_str = ", ".join(cluster)
            dict_[exemplar]=[f'{(exemplar, cluster_str)}']
        return dict_
# Print out values from two columns that have uncommon values
def print_att(df,one,two):
    for i in range(len(df)):
        Str1 = str(df[f'{one}'][i])
        Str2 = str(df[f'{two}'][i])
        Partial_Ratio = fuzz.token_set_ratio(Str1,Str2) 
        if Partial_Ratio<100:
            print(f"{str(df[f'{one}'][i])}, {str(df[f'{two}'][i])}")
            
            
def mult_func_two(df,one,two):
    new=pd.DataFrame()
    new['package_new'] = df[[    f'{one}',    f'{two}']].apply(lambda x: ','.join(x.dropna()), axis=1)
    print(len(new['package_new'].explode().value_counts()))
    print(new['package_new'].explode().value_counts()[0:500])
def mult_func_three(df,one,two,three):
    new=pd.DataFrame()
    new['package_new'] = df[[    f'{one}',    f'{two}',    f'{three}']].apply(lambda x: ','.join(x.dropna()), axis=1)
    print(len(new['package_new'].explode().value_counts()))
    print(new['package_new'].explode().value_counts()[0:500])
def mult_func_four(df,one,two,three,four):
    new=pd.DataFrame()
    new['package_new'] = df[[    f'{one}',    f'{two}',    f'{three}',f'{four}']].apply(lambda x: ','.join(x.dropna()), axis=1)
    print(len(new['package_new'].explode().value_counts()))
    print(new['package_new'].explode().value_counts()[0:500])



#######################################################################    Color    ##########################################################################
colorSplitRegex = r'-| |/|\\|,'
# G E T   A   C O L O R   N A M E
# get the closest color from an rgb tuple
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]
# get the color name either from webcolors or closest_color
def get_colour_name(requested_colour):
    try:
        closest_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
    return closest_name.title()
# G E T   A   C O L O R   V A L U E
# convert a string to an rgb tuple
def parseColor(string):
    try:
        # convert the color string to rgb, default spec is css3
        return webcolors.name_to_rgb(string)
    except Exception:
        pass
# parse raw data for strings we can match on
# if the string has a dash, slash, or space try each color individually
# and average out the RGB values with a simple a+b+c/n forumla
def parseColorData(data):
    # init an empty array for color values
    colorValues = []
    # split it on dashes, spaces, slashes, or commas
    if re.split(colorSplitRegex, data):
        for color in re.split(colorSplitRegex, data):
            colorValues.append(parseColor(color))
    # if it can't be split just full send it
    else:
        colorValues.append(parseColor(color))
    # remove None values
    colorValues = [
        colorValue for colorValue in colorValues if colorValue is not None]
    if not len(colorValues):
        return data
    # find the average color and convert it to a color name
    #######################################################
    # init an average color value
    averageColor = [0, 0, 0]
    for colorValue in colorValues:
        for index, value in enumerate(colorValue):
            averageColor[index] += value
    averageColor = [math.floor(value / (len(colorValues) or 1))
                    for value in averageColor]
    # just change this to a return to run this function
    return get_colour_name(averageColor)





#######################################################################    Clean/Merge Data    ##########################################################################

# NATURAL SORT
######################################################################
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

# i finds the first digit: if it equals zero then there is not a digit, if it equals the end then leave it
def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

# sort the number naturally so 5 will come before 15
def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))
# x['matches']= x['matches'].apply(lambda x: sorted(x, key=natural_key))



#------------atof and natural_keys are complementary-------------
# atof and natural_keys: Will naturally sort 5 infront of 15
def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]
#########################################################################





# REPLACE
#########################################################################
# # Replace specific text in a string: Exmapel1  to  Example1
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# replacement={'Exmapel1':'Example1',    }
# df= df.apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)


def replace_all(text, dic):
    '''
    Given a text and replacement dictionary, performs multiple str.replace()
    Arguments:
    ==========
    text: str, text to perform str.replace()
    dic: dict, where each value is the string to replace, and each key is the string to replace with
    '''
    for key, value in dic.items():
        for v in value:
            text = text.replace(v, key)
    return text
# power['min_temp_sj']=power['min_temp_sj'].apply(lambda x: replace_all(x,{'':['Ã‚Â°F']}))


# Replace all letters and some special characters with a blank
# data['max']=data['max'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(measure)))
#########################################################################




# FRACTIONS
#########################################################################
# Convert a fraction that does not have an int in front of the fraction "1/3"
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
     
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))
#########################################################################





# CONVERTING FRACTIONS, WHOLE NUMBERS, AND NUMBERS WITH LETTERS
#########################################################################
#Converts the number to a decimal if it has a decimal number but leaves off .0 if it is an int. Be sure to conver it back to a string and add -"in"
def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()
# from decimal import *
# TWOPLACES = Decimal(10) ** -2
# tubes_new['height_new']=tubes_new['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))



 
# converts the value to a float if possible and skips the value if not   
def convert_float(val):
    try:
        return float(val)
    except:
        return str(val)
    
    
    

# Definition to round a number
def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x   
    
# Excel might convert 8-1/2 to 08/01/2002 and we need to conver this back into 8.5  
def date_to_decimal(numbers):
    if numbers.find('/') > -1:
        try:
            numbers=str(numbers)
            one, two,three = numbers.split('/')
            tmp = float(two)/float(three[-1])
            num = int(one) + tmp
            return str(num)
        except:
            return numbers
    else:
        return numbers

# Convert a fraction "1/4" that is written as a date "1-4-2021"
def remove_basic_date(numbers):
    if numbers.find('-2021') > -1:
        first, second, year = numbers.split('-')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
    

# Convert a fraction "1/4" that is written as a date "2021-01-04"
def remove_date_year_first(numbers):
    if numbers.find('2021-') > -1:
        year, first, second = numbers.split('-')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
# Convert a fraction "1/4" that is written as a date "1/4/2021"    
def remove_date_year_last(numbers):
    if numbers.find('/2021') > -1:
        first, second,year = numbers.split('/')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
    
regex_year_month_day = r"(?i)(\d\d\d\d\s\d\d\s\d\d)|()"   

def re_extract(pattern, txt): #regex
    try:
        matches = re.findall(pattern, txt)
        tmp_matches = []
        for match in matches:
            for tup in match:
                year, first, second = tup.split(' ')
                num=float(first)+float(second)/(float(year[2:4]))

                return str(num)
                if tup != '':
                    tmp_matches.append(tup)
        return list(set(tmp_matches))
    except:
        return txt



# Just merging everything into one function
def final_done(df,col):
    return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)|#','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda x: remove_basic_date(x)).apply(lambda x: remove_date_year_first(x)).apply(lambda x: remove_date_year_last(x)).apply(lambda measure: re.sub(r'(?<=\d)\s?-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: remove_fraction(x)).apply(lambda x: remove_frac(x)).apply(lambda x: re_extract(regex_year_month_day,x)).apply(lambda x: round_string_float(x))
#     return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda measure: re.sub(r'\-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: round_string_float(x))
    
def final_done(df,col):
    return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)|#','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda x: remove_basic_date(x)).apply(lambda x: remove_date_year_first(x)).apply(lambda x: remove_date_year_last(x)).apply(lambda measure: re.sub(r'(?<=\d)\s?-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: remove_fraction(x)).apply(lambda x: remove_frac(x)).apply(lambda x: re_extract(regex_year_month_day,x)).apply(lambda x: round_string_float(x))
#     return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda measure: re.sub(r'\-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: round_string_float(x))
    
def multi_conversion(numbers):  
    try:
        if (numbers.find('"') > -1 or numbers.find('in') > -1) and numbers.find('ft') > -1:
            try:
                numbers=re.sub(r'ft\.','ft',str(numbers))
                numbers=re.sub(r'in\.','in',str(numbers))
                ft,inch=numbers.split('ft')
                ft=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(ft))
                inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(inch))
                ft_num=float(ft)*12
                total=ft_num+float(inch)
                return str(round_string_float(total))+' in'
            except:
                return str(numbers)
    except:
        return numbers
                       
    if numbers.find('yd') > -1:
        try:
            yd=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(numbers))
            new_yd=individual_value(yd)
#             new_yd=float(new_yd)*12*3
            return str(round_string_float(new_yd))+' yd'
        except:
            return numbers
        
    elif numbers.find('ft') > -1 or numbers.find("'") > -1 :
        try:
            ft=re.sub(r"yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\'|�|•|∩┐╜|\+|°|",'',str(numbers))
            new_ft=individual_value(ft)
#             new_ft=float(new_ft)*12
            return str(round_string_float(new_ft))+' ft'
        except:
            return numbers
        
    elif numbers.find('"') >-1:
        try:
            inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' in'
        except:
            return numbers
    elif numbers.find('in') >-1:
        try:
            inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' in'
        except:
            return numbers
        
    elif numbers.find('mm') >-1:
        try:
            inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' mm'
        except:
            return numbers
        
    elif numbers.find('m') >-1 or numbers.find('M') >-1:
        try:
            inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' m'
        except:
            return numbers
    
    elif numbers.find('gal') >-1:
        try:
            inch=re.sub(r'gal\.|oz\.|yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' gal'
        except:
            return numbers
    elif numbers.find('oz') >-1:
        try:
            inch=re.sub(r'min\.|hr\.|gal\.|oz\.|yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch)matches)+' oz'
        except:
            return numbers
    else:
        return numbers
    
def individual_value(one):
    new_one=round_string_float(re_extract(regex_year_month_day,(remove_frac(remove_fraction(date_to_decimal(re.sub(r'(?<=\d)\s?-',' ',str(remove_date_year_last(remove_date_year_first(remove_basic_date(re.sub(r'00:00:00|00:00|0:00','',str(re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)','',str(one)))).rstrip())))))))))))
    return new_one
#########################################################################


# TEMP RANGES
#########################################################################
# Converts Above 40 Degrees F  and Above 0
def above_freezing(numbers): 
    if numbers.find('above freezing') > -1:   
        numbers=32
        return numbers
    elif numbers.find('above ') > -1:   
        numbers=re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(numbers))
        return numbers
    elif numbers.find('Greater ') > -1:   
        numbers=re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(numbers))
        return numbers
    else:
        return numbers
# data['min']=data['min'].apply(lambda x: above_freezing(x))
    
#Splits temp ranges: 60 to 100 Degrees F, -30 to 200 Degrees F, 50-100 Degrees
def split_height(data, col, dimension):
    data[col]=data[col].apply(lambda x:str(x).lower()) 
    data[['min_'+dimension, 'max_'+dimension]]=data[col].str.split('to',expand=True,)
    data['min_'+dimension]=data['min_'+dimension].apply(lambda x: above_freezing(x)).apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure)))
    data['max_'+dimension]=data['max_'+dimension].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°','',str(measure)))   
    
def split_temp(data, col, dimension):
    data[col]=data[col].apply(lambda x:x.lower())
    data[['min_'+dimension, 'max_'+dimension]]=data[col].str.split(' to |(?<=\d)\s?-|max|below|less(?=u)',expand=True,)
    data['min_'+dimension]=data['min_'+dimension].apply(lambda x: above_freezing(x)).apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•|°| ','',str(measure)))
    data['max_'+dimension]=data['max_'+dimension].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°','',str(measure)))

#########################################################################

def add_unit(df, col, unit):
    df[col] = df[col].fillna('').astype(str) + unit
    df.loc[df[col] == unit, col] = ''
    
def remove_metric(numbers, inputs):
    if str(numbers).find(inputs) > -1:
        numbers=''
        return numbers
    else:
        return numbers
    
from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

In [4]:
f_path=r'G:\Shared drives\GroupBy Public\Customer Success\.Enrich\ScrapedData\Motion - Syndication Deliverable\Final Delivery\_previously_sent\delivered_unified'
csvs = sorted([file for file in os.listdir(f_path) if file[-len('csv'):] == 'csv'])

In [5]:
electrical = pd.read_csv(f'{f_path}/{csvs[6]}', dtype=str)
power=electrical
power=electrical
power=power.dropna(thresh=5,axis=1)
print(len(power.columns.to_list()))
power.columns.to_list()[100:150]

384


['Eyebolt_Type',
 'Features',
 'Finish',
 'Fits_Brand',
 'Fits_Wire_Dia',
 'Flame_Retardant',
 'Flame_Type',
 'Flammable_Non_Flammable',
 'Flash_Point',
 'Flashlight_Battery_Size',
 'Flashlight_Battery_Type',
 'Flashlight_Beam_Type',
 'Flashlight_Body_Material',
 'Flashlight_Grade',
 'Flashlight_Grouping_Type',
 'For_Use_On_Surface_Material',
 'Form',
 'Freezing_Point',
 'Freezing_Point_F',
 'Fuel_Pressure',
 'Function',
 'Gas_Service',
 'Gauge_Conductor',
 'Grade',
 'Green_Certification_or_Other_Recognition',
 'Green_Environmental_Attribute',
 'Grind_Mode',
 'Grip_Style',
 'HP',
 'Handle_Color',
 'Handle_Material',
 'Handle_Type',
 'Headgear',
 'Height',
 'Holds',
 'Hole_Center_to_End',
 'I_D_After_Shrinking',
 'Impact_Rated',
 'Indoor_Outdoor',
 'Inlet_Connection',
 'Inside_Dia',
 'Insulated',
 'Insulation',
 'Insulation_Color',
 'Insulation_Type',
 'Interface',
 'Item_Shim_Stock',
 'Jaw_Capacity',
 'Keyway_Depth',
 'Keyway_Width']

In [6]:
from decimal import *
TWOPLACES = Decimal(10) ** -2
# https://stackoverflow.com/questions/21903842/how-do-i-compare-two-strings-in-python

# Definition to round a number
def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

measure='This is a fraction: 100 1/64'
[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

['This is a fraction: 100.02']

In [13]:
measure='This is a fraction: 1/2'
measure='This is a fraction: 1-1/2'
measure='This is a fraction: 1 1/2'
measure='This is a fraction: 100 1/8'
# measure='This is a fraction: 100 1/64'
measure='.32'
# measure='045'
remove_exponent(Decimal(measure).quantize(TWOPLACES))
re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)

Decimal('0.32')

In [ ]:
.apply(lambda measure: [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)])

In [35]:
# FIND EVERY UNIT OF MEASUREMENT
# https://www.adducation.info/how-to-improve-your-knowledge/units-of-measurement/
# meter
# milimeter
# centimeter
# decimeter
# kilometer
# inch
# foot
# yard
# mile
# square meter
# square feet
# square inch
# square yard
# Acre
# cubic meter
# cubic ft
# cubic inches
# liter
# acre-foot
# fluid ounce fl oz
# weight ounce wt oz
# quart 
# gallon
# cup

# second
# minute
# hour
# day
# year

# grams
# kg
# ounce
# pound
# ton
# newton meter
# watt
# horsepower
# bar
# pascal
# lbs per square inch

# farenheight
# celcius

# Amp
# Coulmb
# volt


In [36]:
# UNICODE FIXES
# https://ftfy.readthedocs.io/en/v4.4.1/#cust_applied
# pip install ftfy

In [37]:
print(fix_text('Broken text&hellip; it&#x2019;s ﬂubberiﬁc!',normalization='NFKC'))
print(fix_text('Broken text&hellip; it&#x2019;s ﬂubberiﬁc!',normalization='NFC'))
print(fix_text('Broken text&hellip; it®|™|Ã|¯|Â|¿|Å|“s ﬂubberiﬁc!',normalization='NFC'))

Broken text... it's flubberific!
Broken text… it's flubberific!
Broken text… it®|™|Ã|¯|Â|¿|Å|"s flubberific!


In [38]:
from ftfy import fix_encoding
print(fix_encoding('®|™|Ã|¯|Â|¿|Å|“'))

®|™|Ã|¯|Â|¿|Å|“


In [29]:
# CREATING A FINAL DEFINITION

In [62]:
x=[
'Diameter',
'Fits_Belts_Up_To_In',
'Fits_Chest_Size',
'Fits_Hat_Size',
'Fits_Shaft_Dia',
'Fits_Shoe_Size',
'Fits_Waist_Sizes',
'Fitting_Material',
'Flavor',
'Fluids_Absorbed',
]

In [53]:
def whole_enchilada(sauce):
    original_col=sauce
    attribute = original_col.lower().replace(' ','_').replace('-','_')
    col = f'Q:{attribute}'
    
    col2=col+'_mm'
    power[col2] = power[power[ original_col].str.contains(r'mm|m|M|ml|mL',na=False)][ original_col]
    power[col2]=power[col2].apply(lambda x: re.sub(r'in.','in',str(x))).apply(lambda x: re.sub(r'ft.','ft',str(x))).apply(lambda x: re.sub(r'oz.','oz',str(x))).apply(lambda x: re.sub(r'gal.','gal',str(x))).apply(lambda x: re.sub(r'dia.','dia',str(x))).apply(lambda x: re.sub(r'yr.','yr',str(x))).apply(lambda x: re.sub(r'hr.','hr',str(x))).apply(lambda x: re.sub(r'yd.','yd',str(x))).apply(lambda x: re.sub(r'g/l.','gl',str(x))).apply(lambda x: re.sub(r'cu.','cu',str(x))).apply(lambda x: re.sub(r'sec.','sec',str(x))).apply(lambda x: re.sub(r'lb|LB|Lb|lB.','lb',str(x))).apply(lambda x: re.sub(r'sq.','sq',str(x))).apply(lambda x: re.sub(r'min.','min',str(x))).apply(lambda x: re.sub(r'max.','max',str(x)))
#     power[col2].apply(lambda x: multi_conversion(x))   
    
    col3=col+'_mL'
    power[col3] = power[power[ original_col].str.contains(r'mL|ml',na=False)][ original_col]
    power[col3]=power[col3].apply(lambda x: re.sub(r'in.','in',str(x))).apply(lambda x: re.sub(r'ft.','ft',str(x))).apply(lambda x: re.sub(r'oz.','oz',str(x))).apply(lambda x: re.sub(r'gal.','gal',str(x))).apply(lambda x: re.sub(r'dia.','dia',str(x))).apply(lambda x: re.sub(r'yr.','yr',str(x))).apply(lambda x: re.sub(r'hr.','hr',str(x))).apply(lambda x: re.sub(r'yd.','yd',str(x))).apply(lambda x: re.sub(r'g/l.','gl',str(x))).apply(lambda x: re.sub(r'cu.','cu',str(x))).apply(lambda x: re.sub(r'sec.','sec',str(x))).apply(lambda x: re.sub(r'lb|LB|Lb|lB.','lb',str(x))).apply(lambda x: re.sub(r'sq.','sq',str(x))).apply(lambda x: re.sub(r'min.','min',str(x))).apply(lambda x: re.sub(r'max.','max',str(x)))
    power[col3].apply(lambda x: multi_conversion(x))  
    
    col4=col+'_cm'
    power[col4] = power[power[ original_col].str.contains(r'cm',na=False)][ original_col]
    power[col4]=power[col4].apply(lambda x: re.sub(r'in.','in',str(x))).apply(lambda x: re.sub(r'ft.','ft',str(x))).apply(lambda x: re.sub(r'oz.','oz',str(x))).apply(lambda x: re.sub(r'gal.','gal',str(x))).apply(lambda x: re.sub(r'dia.','dia',str(x))).apply(lambda x: re.sub(r'yr.','yr',str(x))).apply(lambda x: re.sub(r'hr.','hr',str(x))).apply(lambda x: re.sub(r'yd.','yd',str(x))).apply(lambda x: re.sub(r'g/l.','gl',str(x))).apply(lambda x: re.sub(r'cu.','cu',str(x))).apply(lambda x: re.sub(r'sec.','sec',str(x))).apply(lambda x: re.sub(r'lb|LB|Lb|lB.','lb',str(x))).apply(lambda x: re.sub(r'sq.','sq',str(x))).apply(lambda x: re.sub(r'min.','min',str(x))).apply(lambda x: re.sub(r'max.','max',str(x)))
#     power[col4].apply(lambda x: multi_conversion(x)) 
    col5=col+'_gal'
    power[col5] = power[power[ original_col].str.contains(r'gal',na=False)][ original_col]
    power[col5]=power[col5].apply(lambda x: re.sub(r'in.','in',str(x))).apply(lambda x: re.sub(r'ft.','ft',str(x))).apply(lambda x: re.sub(r'oz.','oz',str(x))).apply(lambda x: re.sub(r'gal.','gal',str(x))).apply(lambda x: re.sub(r'dia.','dia',str(x))).apply(lambda x: re.sub(r'yr.','yr',str(x))).apply(lambda x: re.sub(r'hr.','hr',str(x))).apply(lambda x: re.sub(r'yd.','yd',str(x))).apply(lambda x: re.sub(r'g/l.','gl',str(x))).apply(lambda x: re.sub(r'cu.','cu',str(x))).apply(lambda x: re.sub(r'sec.','sec',str(x))).apply(lambda x: re.sub(r'lb|LB|Lb|lB.','lb',str(x))).apply(lambda x: re.sub(r'sq.','sq',str(x))).apply(lambda x: re.sub(r'min.','min',str(x))).apply(lambda x: re.sub(r'max.','max',str(x)))
#     power[col4].apply(lambda x: multi_conversion(x)) 

    power[col]=power[[
    original_col
    ]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
    # split_height(power,original_col+'_sj','temperature')  
    power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
    power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
    power[col]=power[col].apply(lambda measure: [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)])
    # power[col]=power[col].apply(lambda x:x.strip())
    power[col]=power[col].apply(lambda x: remove_metric(x,"mm"))
    power[col]=power[col].apply(lambda x: remove_metric(x,"mL"))
    power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
    power[col]=power[col].apply(lambda x: remove_metric(x,"gal"))
    # power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
    # power[col]=power[col].apply(lambda x: multi_conversion(x))
    # power[col]=final_done(power,col)
    # add_unit(power,col,' in')
    print(power[col].explode().value_counts())
    print(power[col2].explode().value_counts())
    print(power[col5].explode().value_counts())
#     try:
#         print(power[col1].explode().value_counts())
#         print(power[col2].explode().value_counts())
#         print(power[col3].explode().value_counts())
#         print(power[col4].explode().value_counts())
#         print(power[col5].explode().value_counts())
#     except:

In [31]:
for i in range(len(x)):
    whole_enchilada(x[i])
# whole_enchilada('Fits_Shaft_Dia')

           5915
3 in          9
0.75 in       3
0.5 in        2
0.65625       1
0.53125       1
18 in         1
1.25          1
2 in          1
0.62 in       1
24 in         1
21 in         1
Name: Q:diameter, dtype: int64
     5931
2       6
Name: Q:fits_belts_up_to_in, dtype: int64
          5889
42 in        7
38 in        7
52 in        6
48 in        5
54 in        4
58 in        4
34 in        3
50 in        3
64 in        2
32 in        1
41 in        1
40 in        1
44 in        1
47 in        1
46 in        1
36 in        1
Name: Q:fits_chest_size, dtype: int64
                      5862
0.5                     38
One Size Fits Most      13
0.62                     9
One Size Fits All        6
0.75                     5
0.5 in                   2
6                        2
Name: Q:fits_hat_size, dtype: int64
             5919
0.75 in         2
0.094 in        1
0.156 in        1
0.25 in         1
1.5 in          1
0.400 in        1
0.36 in         1
0.750 in        1
0.55 in 

In [42]:
original_col='Net_Weight'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'-1/8','.12',str(x))).apply(lambda x: re.sub(r'-3/8','.38',str(x))).apply(lambda x: re.sub(r'-1/4','.25',str(x))).apply(lambda x: re.sub(r'-1/2','.5',str(x))).apply(lambda x: re.sub(r'-3/4','.75',str(x))).apply(lambda x: re.sub(r'6-5/8','6.62',str(x))).apply(lambda x: re.sub(r'6-1/2','6.5',str(x))).apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda measure: [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)])
# power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

             2977
14 oz          22
12 oz          20
15 oz          13
10.5 oz        12
1 gal           8
11 oz           6
17 oz           4
5.00 gal        4
13 oz           4
14.00 oz        3
10 lb           2
2.5 lb          2
1.00 gal        2
9.09 oz         2
5 gal           2
20 oz           2
40.48 lb        1
10.85 lb        1
6 oz            1
14.85 lb        1
64.09 lb        1
11.838 lb       1
12.00 oz        1
12.71 oz        1
11.95 lb        1
19.00 oz        1
11.177 lb       1
3 lb            1
11.123 lb       1
10.9 lb         1
14.25 lb        1
13.2 oz         1
3.5 lb          1
55 gal          1
18 oz           1
13.1 lb         1
61.981 lb       1
12.25 lb        1
12.5 lb         1
11.75 oz        1
32 oz           1
11.354 lb       1
11 lb           1
9.65 lb         1
11.65 lb        1
11.104 lb       1
12 lb           1
14.55 lb        1
12.55 lb        1
2.66 lb         1
9.1 lb          1
56.22 lb        1
50 lb           1
14.35 lb        1
16 oz     

In [ ]:
power['Q:color']=power[[
'Color', 
]].apply(lambda x: "".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)

def closest_color(target, colors):
    '''
    Given a list of normalized colors, return the color that the target is closest to
    Modified from: https://bit.ly/3y7BDFn
    '''
    rgb_dict = {i: tuple([v for v in webcolors.name_to_rgb(i)]) for i in colors}
    r, g, b = webcolors.name_to_rgb(target)
    color_diffs = []
    for color in rgb_dict.values():
        try: 
            cr, cg, cb = color
            color_diff = sqrt(abs(r - cr)**2 + abs(g - cg)**2 + abs(b - cb)**2)
            color_diffs.append((color_diff, color))
        except:
            continue
    return [k for k,v in rgb_dict.items() if v==min(color_diffs)[1]][0]

def parseColorData(data):
    if type(data) != str or data=='None' or data =='Color Contrasting Shoulder Leg Straps' or data == 'Color Contrasting Shoulder Chest Leg Seat Straps' or data == 'Indoor/Outdoor Mirror':
        return 'n/a'
    # Clean unwanted text
    elif re.search(r'''(?i)^clear$''',data):
        return 'Clear'
    elif data == 'Navy (Gray)':
        return 'Gray'
    elif data == 'Navy Western':
        return 'Blue'
    elif data == 'Hi-Viz Lime Yellow' or data == 'Lime Yellow':
        return ['Green','Yellow']
    elif data == 'Black,Matte Black Graphite Pattern':
        return 'Black'
    elif data == 'Fluorescent Orange-Red':
        return 'Red'
    elif data =='Multi' or data == 'Multi-Color (Plug), Purple (Cord),Multicolor,Purple':
        return 'Multicolor'
    data = re.sub(r'''(\son\s|\sw\/\s)''',",",data)
    data = re.sub(r'''(?i)(S?s?afety\s?|Welting|Drab|Polyester|Hi(gh)?\-Visibility |Matte | with 5\.0 IR Filter|Metallic|Matte Black Graphite Pattern|Mirror|Infinity|Overedge|Front|Back|Hi(gh)?\-Vis|Tinted|IR\s?Shade\s?\d|Lexan\-Smoke Tinted \(front\)|Handle|Fluorescent|Shade \d(\.\d)?\s?(IR)?|Neon|Moss|\(\w{1,}\)|(out|in)door|Hi[\-\s]?Viz |Hi(gh)?[\-\s]?visibility|clear (?<!\w{1,}$)|light|dark| |Pale|"|')''','',data)

    # Normalized Colors, expand as needed
    normalized_colors=[
        'Black',
        'Blue',
        'Brown',
        'Gold',
        'Gray',
        'Green',
        'Orange',
        'Pink',
        'Purple',
        'Red',
        'Silver',
        'Tan',
        'White',
        'Yellow',
        'Camoflauge'
    ]
    # Replace exotic cases, expand as needed
    data = replace_all(data, {
        'Red':['Maroon'],
        'Clear':['Colorless',' Colourless','Colourless','Clears','Transparent'],
        'Silver':['Chrome','Antique Pewter','Aluminum','Platinum','StainlessSteel','Polyurethane','Steel'],
        'Black':["Blacks"],
        'Pink':['Peach','Coral'],
        'Blue': ['Blues','Ice Blue','IceBlue','Indigo'],
        'Purple':['Magenta'],
        'Green':['Olive','Turquoise','Lime','Chartreuse'],
        'Gray':['Grays','Limestone','Smoke'],
        'White':['Cloudy',"Beige","Whites",'Pearl'],
        'Yellow':['PacificCitrus','Amber','Cream','Yellows'],
        'Teal':['Greenish-Blue'],
        'Yellow':['SAFETYYELLOW'],
        'Brown':['AgedBronze','Aged Bronze','BrushedNickel','Copper','Espresso','Sandalwood','Coffee','Khaki','Brass'],
        ",":['Plated','and','&', ' to ','Water-'], # For multiple colors
        "":['No','Bright','Photoluminescent','Shade3.0',"[","]",
            'Included','Granite',
            'Translucent','Opaque',
            'Pale','-',
            'Concrete','Reddish','Milky','Reflective','SchoolBus','toWhite','onwhite',
           'Endura','Tile','Liquid','Gloss','STEELBLUE®','Sky','Navy'] # For Errorneous values
    })
    # If data is empty, return n/a
    if data =='':
        return 'n/a'
    
    elif 'Assorted' in data:
        return 'Assorted'

    # If data has / or to, return Assorted
    elif re.search('''(\/)''',data):
        if len(list(set(re.findall(r"[\w]+",data))))<3:
            return list(set(re.findall(r"[\w]+",data)))
        elif len(list(set(re.findall(r"[\w]+",data))))>=3:
            return 'Multicolor'
    # If data has a comma, check for duplicates, else Assorted
    elif re.search('''(\,)''',data):
        list_ = list(set([x.capitalize() for x in data.split(',') if x]))
        if len(list_)==1:
            return list_[0]
        elif len(list_)==2:
            
            return list_
        else:
            return 'Multicolor'
    # If data is a normalized color, return color--Closest color
    elif data in normalized_colors:
        return data
    
    # Try find closest normalized colors, else pass
    else:
        try:
            return closest_color(data, normalized_colors)
        except:
            return data

In [ ]:
Counter([str(x) for x in power['Q:color'].apply(lambda x: (parseColorData(x),x))])
power['Q:color'] = power['Q:color'].apply(lambda x: parseColorData(x)).apply(lambda x: 'Natural' if x == 'natural' else x).apply(lambda x: x[0] if (type(x)==list)&(len(x)==1) else x)
Counter([str(x) for x in power['Q:color']])
power['Q:color'].explode().value_counts()

In [34]:
original_col='Fits_Hat_Size'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
col2=col+'_cm'
power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'-1/8','.12',str(x))).apply(lambda x: re.sub(r'-3/8','.38',str(x))).apply(lambda x: re.sub(r'-1/4','.25',str(x))).apply(lambda x: re.sub(r'-1/2','.5',str(x))).apply(lambda x: re.sub(r'-3/4','.75',str(x))).apply(lambda x: re.sub(r'6-5/8','6.62',str(x))).apply(lambda x: re.sub(r'6-1/2','6.5',str(x))).apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda measure: [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)])
# power[col]=power[col].apply(lambda x:x.strip())
power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

                      5862
6.5 to 8                38
One Size Fits Most      13
One Size Fits All        6
6.62 to 7.5              5
6.75 to 7.38             4
6.62 to 7.75             4
6 to 7.12                2
6.5 in to 8 in           2
6.75 to 8                1
Name: Q:fits_hat_size, dtype: int64


In [ ]:
original_col='terminal_category'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='Features_Benefits'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ProductURL'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_Classification'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_Compliant'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_Cut_Grams'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_Dust_Splash_Rating'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_Electrical_Class'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_ISEA_Abrasion_Cycles'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_ISEA_Abrasion_Level'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_ISEA_Cut_Level'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col,'ANSI_ISEA_Cut_Level_2011','ANSI_ISEA_Cut_Level_Palm_Side'
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
power[col]=power[col].apply(lambda x: re.sub(r'2__A2|2','A2',str(x))).apply(lambda x: re.sub(r'A3__3|3','A3',str(x))).apply(lambda x: re.sub(r'1','A1',str(x))).apply(lambda x: re.sub(r'AA','A',str(x)))
power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_ISEA_Cut_Load'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col,'ANSI_ISEA_Cut_Load_Back_Side'
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
power[col]=power[col].apply(lambda x: re.sub(r',','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_ISEA_Needlestick_Force_Fingertips'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col,'ANSI_ISEA_Needlestick_Force_Palm_Side','ANSI_ISEA_Needlestick_Level_Fingertips','ANSI_ISEA_Needlestick_Level_Palm_Side'
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'WxH:|W:','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_ISEA_Puncture_Level'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col,'ANSI_Impact_Type'
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
power[col]=power[col].apply(lambda x: re.sub(r'WxH:|W:','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_SRL_Class'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
power[col]=power[col].apply(lambda x: re.sub(r'WxH:|W:','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='ANSI_SRL_Type'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
power[col]=power[col].apply(lambda x: re.sub(r'WxH:|W:','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='Accessory_Mounting_Options'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
power[col]=power[col].apply(lambda x: re.sub(r'WxH:|W:','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())

In [ ]:
original_col='Accessory_Slots'
attribute = original_col.lower().replace(' ','_').replace('-','_')
col = f'Q:{attribute}'
# col2=col+'mm'
# power[col2] = power[power[ original_col].str.contains(r'mm',na=False)][ original_col]#mm|cm|m|M|mL|ml
# power[col2]=power[col2].apply(lambda x: re.sub(r'Lin.+','ft',str(x))).apply(lambda x: re.sub(r'\d.+to','',str(x))).apply(lambda x: re.sub(r'-.+','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col2].apply(lambda x: multi_conversion(x))   

power[col]=power[[
original_col
]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# split_height(power,original_col+'_sj','temperature')  
power[col]=power[col].apply(lambda x: re.sub(r'®|™|Ã|¯|Â|¿|Å|“','',str(x))).apply(lambda x: re.sub(r'yd\.','yd',str(x))).apply(lambda x: re.sub(r'yr\.','yr',str(x))).apply(lambda x: re.sub(r'dia\.','dia',str(x))).apply(lambda x: re.sub(r'\(\d\d?\)','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'hrs?\.','hr',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'cu\.','cu',str(x))).apply(lambda x: re.sub(r'sec\.','sec',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'sq\.','sq',str(x))).apply(lambda x: re.sub(r'lb\.|LB|lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'cu\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'sq\. ft\.','sq ft',str(x))).apply(lambda x: re.sub(r'min\.','min',str(x))).apply(lambda x: re.sub(r'hr\.','hr',str(x))).apply(lambda x: re.sub(r'gal\.','gal',str(x))).apply(lambda x: re.sub(r'oz\.','oz',str(x))).apply(lambda x: re.sub(r'\([A-Z][A-Z]?\)','',str(x)))      
# power[col]=power[col].apply(lambda x: re.sub(r'sq.+','sq ft/gal',str(x))).apply(lambda x: re.sub(r'and.+','ft',str(x))).apply(lambda x: re.sub(r'to.+','',str(x))).apply(lambda x: re.sub(r'.+-','',str(x))).apply(lambda x: re.sub(r',.+','',str(x))).apply(lambda x: re.sub(r'#.+','',str(x)))
# power[col]=power[col].apply(lambda x: re.sub(r'1-3/4','1.75',str(x))).apply(lambda x: re.sub(r'3/4','0.75',str(x))).apply(lambda x: re.sub(r'2-1/2','2.5',str(x))).apply(lambda x: re.sub(r'2-1/4','2.25',str(x))).apply(lambda x: re.sub(r'5/16','0.62',str(x))).apply(lambda x: re.sub(r'3-1/2','3.5',str(x))).apply(lambda x: re.sub(r'3-1/4','3.25',str(x)))
power[col]=power[col].apply(lambda x: re.sub(r'WxH:|W:','',str(x)))
power[col]=power[col].apply(lambda x:x.strip())
# power[col]=power[col].apply(lambda x: remove_metric(x,"cm"))
# power[col]=power[col].apply(lambda x: multi_conversion(x))
# power[col]=final_done(power,col)
# add_unit(power,col,' in')
print(power[col].explode().value_counts())